<a href="https://colab.research.google.com/github/zachroyer/OMR-CNN/blob/main/HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Part 1: Load/Analyze Data

##What is your dataset? Please describe your dataset and where you got it

TODO - ANSWER HERE

##Load your dataset / Implement your dataloader

In [ ]:
# TODO - load your dataset and set up your dataloader
# If applicable, please include your data with your final submission
# along with instructions for how to load it

# TODO - if not already done, please make a train/val/test split of your dataset

##Dataset Analyis Statistics

In [ ]:
# TODO - code for collecting statistics from your dataset / plots and analysis

TODO - Provide the analysis and statistics of your data

# Part 2: Literature Review

TODO - Follow instructions in the assignment for your literature review

# Part 3: Implement networks in PyTorch

Note of warning here: Depending on how easily you can allocate GPU resources, you may want to make your network much shallower so that you can train it more easily
Aim to have one gradient update take no more than a few seconds
May also want to reduce the number of training steps if training is too slow

In [ ]:
# TODO - implement your PyTorch Module

In [ ]:
# TODO - implement your training loop

In [ ]:
# TODO - plot train and validation accuracy of your model during training

Hint: look at earlier homeworks / resuse code from those to help you here

# Part 4: Run Hyperparameter Experiments

In [ ]:
# TODO - perform hyperparameter grid searches and plot accuracies

In [ ]:
# TODO - perform other experiments and plot accuracies

In [ ]:
# TODO - calculate the final test accuracy

TODO - Explain your experiments above. See assignment doc for more details.